In [13]:
import geopandas as gpd
import folium

In [21]:
wse_gdf = gpd.read_file(r'Z:\js\ResMap\py\output\02yrExistingConditions\tempfiles\ras_wse.shp')
wse_gdf.to_crs(epsg=4326)
# Compute depth as [max_wse - min_elev]
wse_gdf['depth_max'] = wse_gdf['wse_max'] - wse_gdf['min_elev']
wse_gdf['depth_max'] = wse_gdf['depth_max'].round(2)
wse_gdf.shape

(10436, 105)

In [10]:
max_wse_gdf = wse_gdf[['geometry', 'min_elev', 'wse_max', 'depth_max']]
max_wse_gdf.shape

(10436, 4)

In [11]:
max_wse_gdf = max_wse_gdf[max_wse_gdf['depth_max'] > 0]
max_wse_gdf.shape

(535, 4)

In [18]:
f = folium.Figure(width=800, height=550)
m = folium.Map(location=[30.87, -93.29],tiles='cartodbpositron', zoom_start=13, min_zoom = 10, max_zoom=15).add_to(f)

In [19]:
wse_style = lambda x: {
    "color": "#E5214E",
    "fill": False,
    "opacity": 1
}

In [20]:
g = folium.GeoJson(data=max_wse_gdf, style_function=wse_style).add_to(m)
folium.GeoJsonTooltip(
    fields=["depth_max", "wse_max", "min_elev"],
    aliases=["Depth:", "WSE:", "Elevation:"]
).add_to(g)

m